In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn import linear_model
from numpy import genfromtxt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier   
from sklearn.decomposition import PCA
from sklearn import tree
from imblearn.over_sampling import ADASYN 
from collections import Counter
from fancyimpute import KNN,BiScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier  
from sklearn import svm
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from  numpy  import   array
from  numpy  import  argmax
from  sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score as AUC
from  sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import  metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import confusion_matrix as CM,precision_score as P,recall_score as R
from sklearn.preprocessing import Normalizer

In [2]:
data = pd.read_csv('liver.csv')
data.shape

(364, 43)

In [3]:
data.head()

,Age,Weight,Gender,Witnessed Arrest,Bystander CPR,time to ROSC,Epinephrine dose,TTM,Non-cardiac Etiology,Non-shockable Rhythm,...,PLT,Proteins,Glu,pH,PO2,PCO2,MAP,APACHE II,SOFA,outcome
0,50,65,1,1,1,25,5,1.0,0,0,...,338,6.9,439,7.17,413.0,45,60.0,28,16,0
1,66,65,1,1,0,10,4,1.0,0,1,...,158,6.5,160,7.12,84.0,39,75.0,24,11,0
2,57,75,1,1,0,12,3,1.0,0,0,...,216,6.5,356,7.31,68.0,36,140.0,24,10,0
3,54,80,0,1,1,15,5,1.0,1,0,...,142,6.5,157,7.43,83.0,34,103.0,25,12,0
4,68,90,1,1,1,8,2,1.0,0,0,...,200,6.2,170,7.42,113.0,39,142.0,35,8,0


In [4]:
data.isnull().sum()
pd.set_option("display.max_rows", 1000)
print(data.isnull().sum())

Age                        0
Weight                     0
Gender                     0
Witnessed Arrest           0
Bystander CPR              0
time to ROSC               0
Epinephrine dose           0
TTM                        1
Non-cardiac Etiology       0
Non-shockable Rhythm       0
Corticoids                 0
Chronic Anticoagulation    0
Chron Heart Failure        0
Hypertension               0
Coronary heart disease     0
Diabetes                   0
COPD/Asthma                0
Previous Neurol Disease    0
Chronic Renal Failur       0
Liver Cirrhosis            0
Lact                       0
CRP                        1
Creat                      0
ScvO2/SvO2 Admission       0
AST                        0
ALT                        0
LDH                        0
PAL                        0
GGT                        5
TBil                       0
aPTT                       0
PT                         0
INR                        0
PLT                        0
Proteins      

In [5]:
data['TTM'] = data['TTM'].fillna(data['TTM'].median())  # median()--->.mode()
data['CRP'] = data['CRP'].fillna(data['CRP'].mean())  # .mean()--->.mode()
data['Proteins'] = data['Proteins'].fillna(data['Proteins'].mean())
data['GGT'] = data['GGT'].fillna(data['GGT'].mean())  
data['MAP'] = data['MAP'].fillna(data['MAP'].mean())  



In [6]:
data.isnull().sum()
pd.set_option("display.max_rows", 1000)
print(data.isnull().sum())

Age                        0
Weight                     0
Gender                     0
Witnessed Arrest           0
Bystander CPR              0
time to ROSC               0
Epinephrine dose           0
TTM                        0
Non-cardiac Etiology       0
Non-shockable Rhythm       0
Corticoids                 0
Chronic Anticoagulation    0
Chron Heart Failure        0
Hypertension               0
Coronary heart disease     0
Diabetes                   0
COPD/Asthma                0
Previous Neurol Disease    0
Chronic Renal Failur       0
Liver Cirrhosis            0
Lact                       0
CRP                        0
Creat                      0
ScvO2/SvO2 Admission       0
AST                        0
ALT                        0
LDH                        0
PAL                        0
GGT                        0
TBil                       0
aPTT                       0
PT                         0
INR                        0
PLT                        0
Proteins      

In [7]:
print(data['outcome'].value_counts())

0    347
1     17
Name: outcome, dtype: int64


In [8]:
data.columns

Index(['Age', 'Weight', 'Gender', 'Witnessed Arrest', 'Bystander CPR',
       'time to ROSC', 'Epinephrine dose', 'TTM', 'Non-cardiac Etiology',
       'Non-shockable Rhythm', 'Corticoids', 'Chronic Anticoagulation',
       'Chron Heart Failure', 'Hypertension', 'Coronary heart disease',
       'Diabetes', 'COPD/Asthma', 'Previous Neurol Disease',
       'Chronic Renal Failur', 'Liver Cirrhosis', 'Lact', 'CRP', 'Creat',
       'ScvO2/SvO2 Admission', 'AST', 'ALT', 'LDH', 'PAL', 'GGT', 'TBil',
       'aPTT', 'PT', 'INR', 'PLT', 'Proteins', 'Glu', 'pH', 'PO2', 'PCO2',
       'MAP', 'APACHE II', 'SOFA', 'outcome'],
      dtype='object')

In [9]:
colume = ['time to ROSC','Epinephrine dose','PAL', 'PT','INR','Proteins','pH','PCO2',
         'SOFA','AST','ALT','Lact','Creat','LDH','Non-shockable Rhythm', 'Chronic Renal Failur','Corticoids','outcome']
data1 = data[colume]
data1.head()

,time to ROSC,Epinephrine dose,PAL,PT,INR,Proteins,pH,PCO2,SOFA,AST,ALT,Lact,Creat,LDH,Non-shockable Rhythm,Chronic Renal Failur,Corticoids,outcome
0,25,5,123,97,1.00,6.9,7.17,45,16,316,285,5.1,1.0,565,0,0,0,0
1,10,4,107,36,1.91,6.5,7.12,39,11,72,32,15.6,2.3,383,1,0,0,0
2,12,3,47,68,1.22,6.5,7.31,36,10,69,50,3.8,1.4,264,0,0,0,0
3,15,5,62,63,1.28,6.5,7.43,34,12,86,74,4.9,1.2,406,0,0,0,0
4,8,2,74,66,1.25,6.2,7.42,39,8,425,312,3.8,0.7,629,0,0,0,0


In [11]:
data1.to_csv('data1.csv')